<a href="https://colab.research.google.com/github/EkaterinaVoloshina/spellchecker_for_pre_1918_russian/blob/main/spellchecker_last_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone 'https://github.com/EkaterinaVoloshina/spellchecker_for_pre_1918_Russian'

Cloning into 'spellchecker_for_pre_1918_Russian'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (120/120), done.
remote: Total 125 (delta 53), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (125/125), 12.49 MiB | 5.93 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [9]:
% cd 'spellchecker_for_pre_1918_Russian/data'

/content/spellchecker_for_pre_1918_Russian/data


In [10]:
! wget 'https://raw.githubusercontent.com/kak-to-tak/Google_rusngram_spellcheck/master/data/error_df.csv'

--2021-05-30 16:11:01--  https://raw.githubusercontent.com/kak-to-tak/Google_rusngram_spellcheck/master/data/error_df.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22911 (22K) [text/plain]
Saving to: ‘error_df.csv’

error_df.csv        100%[===================>]  22.37K  --.-KB/s    in 0s      

2021-05-30 16:11:01 (95.1 MB/s) - ‘error_df.csv’ saved [22911/22911]



In [44]:
! pip install python-Levenshtein
! pip install pybktree

In [11]:
from collections import Counter

In [12]:
vocabulary = open('vocabulary.txt').read()
counts = Counter(vocabulary.split('\n'))

In [13]:
with open('counts.txt', 'a') as f:
    for word in list(counts.keys()):
        if word != '':
          f.write("{} {}\n".format(word, counts[word]))
f.close()

Скачиваем словарь из томов

In [14]:
vocabulary = open('vocabulary.txt', 'r').read()
train = [word for word in vocabulary.split('\n')]

In [45]:
import nltk
from nltk import punkt # для сегментации предложений
from nltk.tokenize import sent_tokenize
import re #для замены ненужных символов
from collections import Counter
import re
from string import punctuation
from sklearn.metrics import f1_score
import sys
sys.path[0:0] = ['../model']
import pybktree
from Levenshtein import editops, distance
import pandas as pd
import re
from string import punctuation
from functools import reduce
from nltk import WordPunctTokenizer
from string import punctuation

wpt = WordPunctTokenizer()

punct = punctuation+'«»—…“”*№–'

In [46]:
from tqdm import tqdm_notebook as tqdm

import numpy as np
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as opt
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
import pandas as pd
import re
from google.colab import files


plt.rcParams['figure.figsize'] = (15.0, 12.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

torch.manual_seed(42)
np.random.seed(42)

In [17]:
class Vocabulary:
    def __init__(self, data):
        self.AUXILIARY = ["^", "<pad>", "$", "<unk>"]
        self.data = data
        self.chars, self.char2idx, self.idx2char = self.fit(data)
        self.words, self.num_words = self.distinct_words(data)
        self.vocab_size = len(self.chars)
        self.pad_len = self.define_pad()
        
    def fit(self, data):
        """Extract unique symbols from the data, make itos (item to string) and stoi (string to index) objects"""
        chars = list(set(list(',-.0123456789í́абвгдеёжзийклмнопрстуфхцчшщъыьэюяіѣѳѵ') + \
                         [char for word in data for char in word if not self.not_russian(word)]))
        chars = self.AUXILIARY + sorted(chars)
        char2idx = {s: i for i, s in enumerate(chars)}
        idx2char = {i: s for i, s in enumerate(chars)}
        
        return chars, char2idx, idx2char
      
    def distinct_words(self, data):
        print('Estimating cospus size')
        counts = Counter(data)
        words = list(set([word for word in data if len(word)>4 and not self.not_russian(word)]))
        num_words = len(words)
        print('Done!')
        print('You can checkout the number of corpus words and the words themselves with commands .num_words, .words')
        return(words, num_words)

    def transform_all(self, words):
        """Transform list of data to list of indices
        Input:
            - data, list of strings
        Output:
            - list of lists of char indices
        """
        return [self.transform_one(word) for word in words]
    
    def transform_one(self, word):
        """Transform data to indices
        Input:
            - data, string
        Output:
            - list of char indices
        """
        return [self.char2idx[char] if char in self.chars else self.char2idx["<unk>"] for char in word.lower()]
    
    def transform_vecs(self, vecs):
        """Transform list of indices to list of data
        Input:
            - list of lists of char indices
        Output:
            - data, list of strings
        """
        return [self.transform_vec(vec) for vec in vecs]
    
    def transform_vec(self, vec):
        """Transform indices to data
        Input:
            - list of char indices
        Output:
            - data, string
        """
        return "".join([self.idx2char[int(idx)] for idx in vec])
      
    def define_pad(self):
      
        word_lens = [len(x) for x  in self.transform_all(self.words)]
        return(max(word_lens))
      
    def not_russian(self, string):
        s = re.sub("[.,:\'-]", '', string)
        charRe = re.compile(r'[a-zA-Z0-9.]')
        st = charRe.search(s)
        return bool(st) or bool(re.search(r'\d', s) or bool(re.search(r'[^a-zа-яё ]+', string)))

In [18]:
V = Vocabulary(train)

Estimating cospus size
Done!
You can checkout the number of corpus words and the words themselves with commands .num_words, .words


Количество токенов:

In [19]:
print(len(train))

178323


Количество уникальных слов = слов в словаре:

In [20]:
V.num_words

130311

Максимальная длина слова, под которую потом будут "подгоняться" остальные:

In [21]:
V.pad_len

29

Тут создается тензор (матрица) для модели:

In [22]:
class CharDataset(Dataset):
    def __init__(self, data, V):
        super(Dataset).__init__()
        self.V = V
        self.chars = V.chars
        self.data = data
        self.pad_len = V.pad_len
        
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        word = self.data[idx]
        indices = self.V.transform_one(word)
        x = torch.ones((self.pad_len), dtype=torch.int64)
        y = torch.ones((self.pad_len), dtype=torch.int64)
        mask = torch.zeros((1,), dtype=torch.int64)
        word_len = min(len(indices), self.pad_len-1)
        for idx_i, i in enumerate(indices[:word_len-1]):
            x[idx_i+1] = i
            
        for idx_i, i in enumerate(indices[:word_len]):
            y[idx_i] = i
            
        mask[0] = word_len+1
            
        x[0] = self.V.char2idx["^"]
        y[word_len] = self.V.char2idx["$"]
        
        
        return x, y, mask
    
    def gen_input_tensor(self, letters):
        indices = self.V.transform_one(letters)
        tensor = torch.ones((self.pad_len), dtype=torch.int64)
        word_len = min(len(indices), self.pad_len)
        for idx_i, i in enumerate(indices[:word_len]):
            tensor[idx_i+1] = i
            
        tensor[0] = 0
            
        return tensor
    
    def resized(self, new_size):
        return CharDataset(data=self.data[:new_size], V=self.V)

Тут прописана архитектура модели: входный слой - скрытый слой - выходной

In [23]:
class CharLM(nn.Module):
    def __init__(self, vocab_size, emb_dim=27, hidden_size=128, n_layers=2, drop_prob=0.5, word_len=27):
        super(CharLM, self).__init__()
        self.vocab_size = vocab_size
        self.word_len = word_len
        self.emb_dim = emb_dim
        self.hidden_size = hidden_size
        self.n_layers=n_layers
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lstm = nn.LSTM(self.emb_dim, hidden_size, num_layers=self.n_layers, batch_first=True) #,dropout=drop_prob)#, bidirectional=True)
        self.output = nn.Linear(in_features=self.word_len*self.hidden_size, out_features=self.vocab_size*word_len)
    
    def forward(self, input_word):
        # torch.Size([64, 27])
        embeded = self.emb(input_word).view(-1, self.word_len, self.emb_dim)
        # torch.Size([64, 27, 27]) or torch.Size([1, 27, 27])
        out, hidden = self.lstm(embeded)
        # torch.Size([64, 27, 1024])
        batch_size, seq_size, hidden_size = out.shape
        out = out.contiguous().view(batch_size, seq_size*hidden_size)
        # torch.Size([64, 13824])
        out = self.output(out).view(-1, self.word_len, self.vocab_size)
        # torch.Size([64, 27, 55])
        # final = F.softmax(out, dim=-1)
        # torch.Size([64, 27,55])
        return out
      
        
class CharLinearModel(nn.Module):
    def __init__(self, vocab_size, emb_dim=27, hidden_size=128, word_len=27):
        super(CharLinearModel, self).__init__()
        self.vocab_size = vocab_size
        self.word_len = word_len
        self.emb_dim = emb_dim
        self.emb = nn.Embedding(embedding_dim=emb_dim, num_embeddings=vocab_size)
        self.hidden = nn.Linear(in_features=self.word_len*self.emb_dim, out_features=hidden_size)
        self.output = nn.Linear(in_features=hidden_size, out_features=vocab_size*word_len)
        
    def forward(self, input_word): 
        # torch.Size([64, 15])
        input_word = self.emb(input_word)
        # torch.Size([64, 15, 15])
        input_word = input_word.view(-1, self.word_len*self.emb_dim)
        # torch.Size([64, 225])
        hidden = F.relu(self.hidden(input_word)) 
        # torch.Size([64, 1024])
        final = self.output(hidden).view(-1, self.word_len, self.vocab_size) 
        # torch.Size([64, 15, 38])
        final = F.softmax(final, dim=-1) 
        # torch.Size([64, 15, 38])
        return final

Тут считается функция потерь

In [24]:
class MaskedLoss(nn.Module):
    def __init__(self, base_criterion=nn.CrossEntropyLoss()):
        super(MaskedLoss, self).__init__()
        self.base_criterion=base_criterion
        
    def forward(self, predict, target, mask):
        batch_size = predict.shape[0]
        loss = []
        for batch_id in torch.arange(batch_size):
            word_len = mask[batch_id][0]
            word_loss = torch.stack([
                self.base_criterion(
                    predict[batch_id:batch_id+1, idx, :],
                    target[batch_id:batch_id+1, idx]
                ) for idx in torch.arange(word_len)
            ])
            
            word_loss = torch.mean(word_loss)
            loss.append(word_loss)
        loss = torch.mean(torch.stack(loss))
        return loss

Создаем тензоры/матрицы из обучающей и тестовой выборки

In [25]:
train_dataset = CharDataset(V.words[:int(0.9*(len(V.words)))], V)
val_dataset = CharDataset(V.words[int(0.9*(len(V.words))):], V)

Еще раз проверим, сколько слов в словаре:

In [26]:
len(train_dataset)

117279

Тут создается модель:

In [27]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model = CharLM(V.vocab_size, word_len=V.pad_len, emb_dim=128, hidden_size=64)
# model = CharLinearModel(V.vocab_size, V.pad_len, 1024, word_len=V.pad_len)
model.to(device)
criterion = nn.CrossEntropyLoss()
loss_func = MaskedLoss(criterion)
optimizer = opt.Adam(lr=0.001, params=model.parameters(), weight_decay=0.0005)
lr_sched = opt.lr_scheduler.ReduceLROnPlateau(optimizer, patience=7, verbose=True, threshold=0.001)

Тут задаются параметры (почему такие, я не знаю):

In [28]:
train_losses = []
train_perplexes = []
train_f1 = []

dev_losses = []
dev_perplexes = []
dev_f1 = []

early_stopping_values = [999]
early_stopping_count = 0
early_stopping_trigger = 7

model_filename = "old_rus_lm.pth"

# dataset_size = 10000
batch_size = 16
n_epochs = 200
train_loader = DataLoader(dataset=train_dataset.resized(2000), batch_size=batch_size, num_workers=8)
val_loader = DataLoader(dataset=val_dataset.resized(200), batch_size=batch_size, num_workers=4)
#train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, num_workers=0)
#val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, num_workers=0)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Тут модель тренируется.
Максимальное число итераций - 200, но с нашим словарем прошла около 10

In [29]:
cel = []
pl = []
for i in range(5):
    model = CharLM(V.vocab_size, word_len=V.pad_len, emb_dim=128, hidden_size=64)
    # model = CharLinearModel(V.vocab_size, V.pad_len, 1024, word_len=V.pad_len)
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    loss_func = MaskedLoss(criterion)
    optimizer = opt.Adam(lr=0.001, params=model.parameters(), weight_decay=0.0005)
    lr_sched = opt.lr_scheduler.ReduceLROnPlateau(optimizer, patience=7, verbose=True, threshold=0.001)
    for ep in range(n_epochs):
        model.train()
        epoch_loss = []
        epoch_perplexes = []
        epoch_f1 = []
        for x, y, mask in tqdm(train_loader, desc='Training'):
            x = x.to(device)
            y = y.to(device)
            word = model(x)
            loss = loss_func(word, y, mask)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            loss_value = loss.detach().cpu().numpy()
            perplexity_value = torch.pow(loss.detach(), 2).cpu().numpy()
            
            epoch_loss.append(loss_value)
            epoch_perplexes.append(perplexity_value)


            
        mean_train_loss = np.mean(epoch_loss)
        mean_train_perplexity = np.mean(epoch_perplexes)

        
            
        val_losses = []
        val_perplexes = []
        val_accuracy = []
        model.eval()
        with torch.no_grad():
            for x, y, mask in tqdm(val_loader, desc='Validating'):
                x = x.to(device)
                y = y.to(device)

                word = model(x)
                loss = loss_func(word, y, mask)

                loss_value = loss.cpu().numpy()
                perplexity_value = torch.pow(loss, 2).cpu().numpy()
                

                val_losses.append(loss_value)
                val_perplexes.append(perplexity_value)
                
        mean_val_loss = np.mean(val_losses)
        mean_val_perplexity = np.mean(val_perplexes)
        
        lr_sched.step(mean_val_loss)
        
        if mean_val_loss <= min(early_stopping_values):
            early_stopping_values.append(mean_val_loss)
            print("Quality improved. Saving Model.")
            torch.save(model.state_dict(), model_filename)
    #         files.download(model_filename)
            
            early_stopping_count = 0
        else:
            early_stopping_count += 1
            early_stopping_values.append(mean_val_loss)
            if early_stopping_count >= early_stopping_trigger:
                print(f"Early stopping triggered, minimal loss reached = {min(early_stopping_values)}")
                break
                cel.append(mean_val_loss)
                pl.append(mean_val_perplexity)
                

        print(f"Epoch {ep}:\n train_loss={mean_train_loss:.5f},"\
              f" train_perplexity={mean_train_perplexity:.5f},\n"\
              f" val_loss={mean_val_loss:.5f},"\
              f" val_perplexity={mean_val_perplexity:.5f}")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



Quality improved. Saving Model.
Epoch 0:
 train_loss=2.63655, train_perplexity=7.15745,
 val_loss=1.79254, val_perplexity=3.22621


...


Early stopping triggered, minimal loss reached = 0.40148231387138367



Early stopping triggered, minimal loss reached = 0.40148231387138367



Early stopping triggered, minimal loss reached = 0.40148231387138367



Early stopping triggered, minimal loss reached = 0.40148231387138367



Early stopping triggered, minimal loss reached = 0.40148231387138367


Сохраняем модель:

In [30]:
files.download(model_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
device = torch.device('cpu')

def seq_prob(model, dataset, vocab):
    for i in range(len(dataset)):
        x, y, word_len = dataset[i]
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)[0]
        probs = []
        output = vocab.transform_vec(y[:word_len])
        for char_idx in range(1,word_len[0]):
            char = output[char_idx]
            if char=='<':
                break
            try:
                idx = vocab.char2idx[char]
            except KeyError:
                idx = vocab.char2idx["<unk>"]
            probs.append(y_pred[char_idx][idx])
        try:
            multiplication = reduce(lambda x, y: x*y, probs)
            final_prob = multiplied/word_len[0].float().to(device)
        except:
            final_prob = 1E-7 # костыль
        return final_prob

In [35]:
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data.dataloader import DataLoader


class CharDataset(Dataset):
    def __init__(self, data, V):
        super(Dataset).__init__()
        self.V = V
        self.chars = V.chars
        self.data = data
        self.pad_len = V.pad_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        word = self.data[idx]
        indices = self.V.transform_one(word)
        x = torch.ones((self.pad_len), dtype=torch.int64)
        y = torch.ones((self.pad_len), dtype=torch.int64)
        mask = torch.zeros((1,), dtype=torch.int64)
        word_len = min(len(indices), self.pad_len - 1)
        for idx_i, i in enumerate(indices[:word_len - 1]):
            x[idx_i + 1] = i

        for idx_i, i in enumerate(indices[:word_len]):
            y[idx_i] = i

        mask[0] = word_len + 1

        x[0] = self.V.char2idx["^"]
        y[word_len] = self.V.char2idx["$"]

        return x, y, mask

    def gen_input_tensor(self, letters):
        indices = self.V.transform_one(letters)
        tensor = torch.ones((self.pad_len), dtype=torch.int64)
        word_len = min(len(indices), self.pad_len)
        for idx_i, i in enumerate(indices[:word_len]):
            tensor[idx_i + 1] = i

        tensor[0] = 0

        return tensor

    def resized(self, new_size):
        return CharDataset(data=self.data[:new_size], V=self.V)

def read_corpus(file_path):
    """ Read file, where each word is dilineated by a `\n`.
    @param file_path (str): path to file containing vocabulary
    """
    vocabulary = open(file_path, 'r').read()
    data = [word for word in vocabulary.split('\n')]

    return data

In [36]:
class spellCorrect(object):
    """A class to correct non-dictionary words in Google Ngrams using Noisy Channel Model, Error Confusion Matrix, Damerau-Levenshtein Edit Distance and a Char Language model to help define real-word non-dictionary tokens
Usage:
Input: 'астроном1я'
Response: астрономія"""
    def __init__(self, V=None, model=None):
        """Constructor method to load external probMaker class, load dictionary and words counts """
        self.vocab = self.load_vocab()
        self.counts = self.load_counts()
        self.trie = pybktree.BKTree(distance, self.vocab)
        self.error_df = self.load_error_df()
        self.pm = probMaker(self.error_df, self.counts)
        self.V = V
        self.model = model
    
    def load_vocab(self):
        """Method to load dictionary from external data file."""
        print ("Loading dictionary from data file")
        vocabulary = open('vocabulary.txt', 'r').read()  # pre-reform word forms
        return list(set([word.lower() for word in vocabulary.split("\n") if len(word)>4]))

    def load_counts(self):
        """Method to load counts from external data file."""
        print("Loading counts")
        counts = {}
        lines = open('counts.txt', 'r').read().split("\n")
        for line in lines:
            if line:
                l = line.split()
                if len(l) > 1:
                    key, value = l[0],l[1]
                    counts[key] = value
        return counts

    def load_error_df(self):
        """Method to load a dataframe containing  from external data file."""
        print("Loading error dataframe")
        error_df = pd.read_csv('error_df.csv', error_bad_lines=False)
        return error_df

    def gen_candidates(self, word):
        """Method to generate set of candidates for a given word using Damerau-Levenshtein Edit Distance of 1 and 2"""
        return self.trie.find(word.lower(), 2)
    
    def get_best(self, error):
        """Method to calculate channel model probability for errors."""
        candidates = self.gen_candidates(error.lower())
        p = [0]*len(candidates)
        for i, candidate_ in enumerate(candidates):
            candidate = candidate_[-1]
            p_ew_candidate = []
            for res in editops(candidate,error):
                editop, w_idx, e_idx = res
                if editop == 'replace':
                    e=error[e_idx]
                    w=candidate[w_idx]
                elif editop == 'insert':
                    e=error[e_idx-1:e_idx+1]
                    w=candidate[w_idx-1]
                elif editop == 'delete':
                    if e_idx!=0:
                        e=error[e_idx-1]
                        w=candidate[w_idx-1:w_idx+1]
                    else:
                        e=error[e_idx]
                        w=candidate[w_idx:w_idx+2]
                else:
                    print(editops(candidate,error))
                    return error 
                p_ew_candidate.append(self.pm.P_ew(editop,e,w))
            if p_ew_candidate:
                p[i] = self.pm.P_w(candidate)*reduce(lambda x, y: x*y, p_ew_candidate)/len(p_ew_candidate)
            else:
                return error
        try:    
            best_idx = p.index(max(p))
            return(candidates[best_idx][-1])
        except ValueError:
            return error
            print(editops(candidate,error))
    
    def to_check(self, string, seqprob, upper_boundary=0, lower_boundary=0):
        '''Rid of non-cyrilic words, 
        words with len < 5, 
        dictionary words 
        and words with probability>? of being a non-error word given by the language model'''

        if len(string)<5:
            return string
        # alphanumeric words stay the same
        s = re.sub("[.,:\'-]", '', string)
        charRe = re.compile(r'[^a-zA-Z0-9.]')
        st = charRe.search(s)

        if not bool(st):
            return string
        else:
            # vocab words stay the same
            if self.trie.find(s.lower(), 0):
                return string
            else:
                if upper_boundary>0 and lower_boundary>0:
                    if not seqprob:
                        dataset = CharDataset([string], V=self.V)
                        seqprob = seq_prob(self.model, dataset, self.V)
                        
                    if seqprob<=upper_boundary or seqprob>=lower_boundary:
                        self.correction = self.return_upper(self.get_best(string),string)
                        return self.correction
                    else:
                        self.correction = self.return_upper(self.get_best(string),string)
                        if self.rules(self.correction, string):
                            return self.correction
                        else:
                            return string
                else:
                    return self.return_upper(self.get_best(string),string)
                
    def return_upper(self,w,e):
        if e.isupper():
            return w.upper()
        else:
            if e[0].isupper():
                return w[0].upper()+w[1:]
            else:
                return w
            
    def rules(self, w, e):
        if len(e)>4:
            rule1 = e[-1] in 'шщцЦШЩ'
            rule2 = any(i.isdigit() for i in e)
            rule3 = e[-2] in 'щшцЩШЦ' and e[-1] in 'июяИЮЯ'
            rule4 = 'ѣ' or 'Ѣ'  in w
            if rule1 or rule2 or rule3 or rule4:
              return True
        return False


In [37]:
import numpy as np
import pandas as pd
from Levenshtein import editops, distance
from collections import Counter
from sklearn.metrics import confusion_matrix

class probMaker(object):
    
    def __init__(self, error_df, counts):
        self.counts = counts
        self.error_df = error_df
        self.sub_mtrx, self.sub_labels = self.confusion_mtrx('replace')
        self.ins_mtrx, self.ins_labels = self.confusion_mtrx('insert')
        self.del_mtrx, self.del_labels = self.confusion_mtrx('delete')
        
    def P_w(self, w):
        '''Calculate P(word)'''
        N = sum(list(map(int,self.counts.values())))
        if w in self.counts:
            return (int(self.counts[w]) + 1) / (N + len(self.counts)) # лаплассовское сглаживание?
        else:
            return 1/(N + len(self.counts))

    def P_ew(self, editop, e, w):
        '''Calculate P(e|w)'''
        
        if editop=='replace':
            return self.check_cofusion_mtrx(self.sub_labels,self.sub_mtrx,e,w)
        elif editop=='insert':
            return self.check_cofusion_mtrx(self.ins_labels,self.ins_mtrx,e,w)
        elif editop=='delete':
            return self.check_cofusion_mtrx(self.del_labels,self.del_mtrx,e,w)
        else:
            return 0.0001 # костыль

    def check_cofusion_mtrx(self,labels,cm,e, w):
        if e in labels:
            if w in labels:
                e_idx = labels.index(e)
                w_idx= labels.index(w)
                return(cm[w_idx, e_idx]/(sum(cm[w_idx,:])))
        else:
            if w in labels:
                w_idx= labels.index(w)
                return(1/(sum(cm[:, w_idx])))
        return(1/(cm.shape[0]*cm.shape[1])) 

    def confusion_mtrx(self, editop):
        '''Building confusion matrices of the checked partition of data'''
        error_df = self.error_df       
        df = pd.concat([error_df[error_df['editop']==editop], \
                        error_df[error_df['editop']=='equal']], axis=0, ignore_index=True)
        correct_tokens = [list(word[:int(df.idxw.iloc[[i]])]) + \
                          [df['e|w'].iloc[[i]][i][df['e|w'].iloc[[i]][i].find('|')+1:]] + \
                          list(word[int(df.idxw.iloc[[i]]) + \
                                    len(df['e|w'].iloc[[i]][i][df['e|w'].iloc[[i]][i].find('|')+1:]):]) \
                          for i, word in enumerate(df['correction'].astype(str).tolist())]
               
        correct_elements = [el for token in correct_tokens for el in token if el]
               
        error_tokens = [list(word[:int(df.idxe.iloc[[i]])]) + \
                        [df['e|w'].iloc[[i]][i][:df['e|w'].iloc[[i]][i].find('|')]] + \
                        list(word[int(df.idxe.iloc[[i]])+len(df['e|w'].iloc[[i]][i][:df['e|w'].iloc[[i]][i].find('|')]):]) \
                        for i, word in enumerate(df['error'].astype(str).tolist())]
        error_elements = [el for token in error_tokens for el in token if el]
        return np.add(1, confusion_matrix(correct_elements, error_elements)),\
               sorted(list(set(correct_elements+error_elements))) # with smoothing


In [38]:
data = read_corpus('vocabulary.txt')
V = Vocabulary(data)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
model = CharLM(V.vocab_size, word_len=V.pad_len, emb_dim=128, hidden_size=64)
#     model = CharLM(V)
model.to(device)
model_filename = "old_rus_lm.pth"
print('Loading a model')
model.load_state_dict(torch.load(model_filename, map_location={'cuda:0': 'cpu'}))
print('Done')
    
correct = spellCorrect(V=V, model=model)

Estimating cospus size
Done!
You can checkout the number of corpus words and the words themselves with commands .num_words, .words
Loading a model
Done
Loading dictionary from data file
Loading counts
Loading error dataframe


In [39]:
correct.to_check('Галнлео', seqprob=False)

'Галилео'

In [47]:
from tqdm.notebook import tqdm

Проверка качества:

In [62]:
file_n = open('unedited.txt').read()

In [63]:
text = file_n.split('\n')

In [64]:
text = text[:1000]

In [69]:
#text = set(wpt.tokenize(text))
print(f'Количество уникальных слов в тексте - {len(text)}')
#corrected = dict()
corrected = []
for i in tqdm(text):
    corrected.append(correct.to_check(i, seqprob=False))

Количество уникальных слов в тексте - 1000


In [74]:
edited = open('edited.txt').read().replace('агинымъ', '').split('\n')
edited = edited[:1000]

In [75]:
from sklearn.metrics import accuracy_score
 
accuracy = accuracy_score(edited, corrected)
accuracy

0.584